# Import Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
tf.__version__

# Load Pre-Built CNN

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
#false- dense layers is not included-only Con and pooling layers are included
#include imagenet weights also

In [ ]:
base_model.summary()

In [ ]:
len(base_model.layers)

In [ ]:
#names = ['mixed3', 'mixed5', 'mixed8', 'mixed9'] #names of some mixed layers in the architecture
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]
layers

In [ ]:
deep_dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)#two output layers mixed3 mixed5
# get outputs from the hidden layers to achieve deep dream images

# Load and Preprocess the Image

In [ ]:
image = tf.keras.preprocessing.image.load_img('../input/deepdream/StaryNight.jpg',
                                              target_size=(225, 375)) #with respect to inception layer
plt.figure(figsize=(15,15))
plt.axis('off')
plt.imshow(image);

In [ ]:
image.mode, len(image.mode), image.size

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image) #convert image to np array for compatibility
type(image), image.shape

In [ ]:
image = tf.keras.applications.inception_v3.preprocess_input(image)   #all preprocessing takes place to be compatible with inception network

# Getting the Activations

In [ ]:
image.shape

In [ ]:
image_batch = tf.expand_dims(image, axis=0)
image_batch.shape

In [ ]:
activations = deep_dream_model.predict(image_batch)
len(activations)

In [ ]:
#activations[0]
activations[0].shape, activations[1].shape #mixed3, mixed5

# Calculating the Loss

In [ ]:
def calculate_loss(image, network):
    image_batch = tf.expand_dims(image, axis=0) #convert one image to batch format
    #activations = network.predict(image_batch)
    activations = network(image_batch)

    losses = []
    for act in activations:
        loss = tf.math.reduce_mean(act) #get average
        losses.append(loss)

    return tf.reduce_sum(losses)

In [ ]:
loss = calculate_loss(image, deep_dream_model)
loss

# Gradient Ascent
## Custom function (Opposite to gradient descent)

In [ ]:
@tf.function #used to define as a global variable

def deep_dream(network, image, learning_rate):
    with tf.GradientTape() as tape: #GradientTape - class in tf to access gradient
        tape.watch(image) #make changes in image
        loss = calculate_loss(image, network)
    
    gradients = tape.gradient(loss, image) #derivative
    gradients /= tf.math.reduce_std(gradients)  #normalizing
    
    image = image + gradients*learning_rate  #making changes to original image repeatedly till the loss is maximum(gradient ascent)
    
    image = tf.clip_by_value(image, -1, 1) #scale values in the range [-1, 1]
    
    return loss, image

In [ ]:
#reverse the preprocessing steps from inception layers to get image after change from gradient ascent

def inverse_transform(image):
    image = 255 * (image + 1.0) / 2.0 #scale from [-1, 1] to original formats
    return tf.cast(image, tf.uint8) #convert pixels to integer values

In [ ]:
def run_deep_dream(network, image, epochs, learning_rate):
    for epoch in range(epochs):
        loss, image = deep_dream(network, image, learning_rate)
        
        if (epoch % 200==0):
            plt.figure(figsize=(15,15))
            plt.imshow(inverse_transform(image))
            plt.show()
            print('Epoch {}, loss {}'.format(epoch, loss))

# Generating Images
## Image 1

In [ ]:
image.shape, type(image)

In [ ]:
run_deep_dream(network=deep_dream_model,
               image=image,
               epochs=8000,
               learning_rate=0.0001
              )

## Image 2

In [ ]:
image = tf.keras.preprocessing.image.load_img('../input/deepdream/sky.jpeg',
                                              target_size = (225, 375))
plt.figure(figsize=(15,15))
plt.axis('off')
plt.imshow(image);

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image)
image = tf.keras.applications.inception_v3.preprocess_input(image)

In [ ]:
run_deep_dream(network = deep_dream_model, image = image, epochs = 8000, learning_rate = 0.0001)